##Install NVIDIA

In [ ]:
!nvidia-smi

##Install Libraries

In [ ]:
#Install necessary libraries
!pip install transformers
!pip install tensorboardX
!pip install sentencepiece

In [ ]:
#Import libraries and initialize the model
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = False

if use_own_model:
  model_name_or_path = "/content/model_output"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""

# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

def to_list(tensor):
    return tensor.detach().cpu().tolist()

# Setup model
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer)
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(
    model_name_or_path, do_lower_case=True)
model = model_class.from_pretrained(model_name_or_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

In [ ]:
# Define functions
def run_prediction(question, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs).values()

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

##Run predictions

In [ ]:
full_text = """Purpose :
The objective of the Employee Referral Programme process document is to provide guidelines necessary to    assist all employees at Techversant who refer talent profiles to the Organization. This program is designed to help source talent, build diversity, and find the right candidates for open positions and to reward our employees for           the right referrals. This programme will encourage all at Techversant to act as talent ambassadors for the Organization. 
Techversant is an equal opportunity employer with a diverse workforce. All referred candidates will be given the same consideration and will pass through the established process and procedures of the Organization.  
Please note that it is not the intent for these guidelines to describe every situation. Management  judgment will / may be necessary in certain sit

Scope :
The process will be applicable for all referrals hired through the employee referral program, effective 1st October 2021.  It covers… All open positions in the Organization that would fall under Employee Referral Programme as announced by the Talent Acquisition Team and as approved by the Management.  • Special referral schemes catering to critical needs as announced by the Talent Acquisition Team and as approved by the management. 

Eligibility for referring candidates :
All employees of Techversant are eligible to refer candidates under the employee referral program. Referrer should not be involved in the hiring process of the referred candidate (viz. prescreening, interview and offer release). • All those directly involved in the hiring process for the referral positions will not be part of this programme.

Process :
 The employee referral process will be as below: o The Talent Acquisition Team will declare the open positions which would come under the employee referral program. It will have the referral category mentioning the bonus amount for the referral positions and the date until which the program is open for the respective referral bonus along with the other relevant position details.  o Only positions announced under the employee referral bonus programme will be considered for processing.  o The Referral Bonus Amount will be notified for the open positions, based on the criticality and priority of the position, availability in the market, the role and designation of the position and the quality of the candidate hired.  • The Referral Bonus amount could range from INR 5000 to INR 50,000 or an amount as decided by the Talent Acquisition Team and approved by the Management based on the open position.  o All referrals are to be sent to the HR Email id or uploaded to a portal, as notified, along with other relevant details.  o Response emails on the progress of referral would be send to the referrers, as and when an activity happens against the referral o Rejected profiles may get considered only after six (6) months, subject to final approval and  discretion of the Management. 

Eligibility for Referral Bonus:
• All employees are eligible to refer candidates under the employee referral programme. • All employees who are active at the time of the employee referral bonus payment date - both referrer and the referee will need to on rolls with Techversant and not serving the notice period. • All who are directly involved in external recruitment are not eligible • The referral bonus programme is applicable only for permanent employee hires and fixed term consultants. • If the referral candidate joins as contractors, the referrer will be eligible for referral bonus only upon their successful conversion to regular rolls of the company.  • To be eligible for referral bonus, the profile has to be sent to the Talent Acquisition team by the employee at the mentioned email id or portal as applicable. Any application processed without the involvement of the Talent Acquisition team will not be eligible for a referral bonus. • In the event a referral is received through multiple referrers, the first referrer becomes eligible for the referral bonus. The Talent Acquisition Team shall intimate the other referrers before initiating the recruitment process. • Referral bonus will not be applicable for fresher recruitment.  • Referral bonus will not be applicable for positions under the Enabling / Support Functions, unless there is an exception as approved by the Management.  

Referral Bonus Payout Details :
• The Referral bonus would be paid in the subsequent month payroll once the selected referral completes minimum 6 months or the probation period, whichever is more, at Techversant and upon successful completion of background verification of the referral, as the case may be.  • Both Referrer and Referral should be on Techversant’s payroll at the time of bonus payout  and not serving the notice period o Bonus payment for the employees on long leave would be credited when the employee returns to active duty. • The referral bonus will be paid along with salary and is a taxable income. Any tax applicable will be deducted at source. 

Authority:
The Management and Human Resources Group reserves the right to revise or eliminate this policy. Any such changes will be communicated.  
All ‘exceptions’ to the statements of this policy document must be appealed the HR Function Head and will be considered on an individual case basis, subject to review and approval from the Management."""

In [ ]:
questions = ["What is Employee Referral Programme ?"]

# Run method
predictions = run_prediction(questions, full_text)

In [ ]:
# Print results
print(list(predictions.values())[0])